In [1]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [2]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

## statsmodels（先頭30フレームの平均）

In [3]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 平均を計算
    mean_values_per_label = first_30_frames_per_label.groupby("label").mean(numeric_only=True)
    
    # NaNを含む行を削除
    mean_values_per_label = mean_values_per_label.dropna(axis=0)
    
    # Yを追加
    mean_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, mean_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.996101  0.621464  0.302809  0.478347  0.316149  0.184034   
answer2     0.997114  0.621363  0.350285  0.505225  0.273179  0.185705   
answer3     0.995380  0.640608  0.414539  0.464368  0.397188  0.157348   
answer4     0.994671  0.688352  0.413224  0.430911  0.422744  0.150436   
answer5     0.994515  0.686247  0.407986  0.458798  0.447200  0.149847   
answer6     0.995428  0.689668  0.524745  0.374945  0.436807  0.159959   
answer7     0.994965  0.625367  0.417123  0.453035  0.481933  0.189774   
answer8     0.994378  0.681734  0.408478  0.559817  0.323617  0.161812   
answer9     0.996479  0.583961  0.364557  0.477847  0.410745  0.205643   
answer10    0.994109  0.693208  0.466591  0.512081  0.494155  0.179663   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
lab

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.999145  0.550915  0.245734  0.533718  0.245004  0.177568   
answer2     0.986787  0.384628  0.281490  0.294987  0.334852  0.107579   
answer3     0.999173  0.815454  0.467734  0.459796  0.437773  0.080840   
answer4     0.999177  0.791023  0.461801  0.567085  0.375046  0.090935   
answer5     0.999433  0.826166  0.703675  0.229415  0.425828  0.076186   
answer6     0.999672  0.735245  0.346020  0.689828  0.425815  0.081355   
answer7     0.999563  0.806141  0.404329  0.543039  0.397249  0.065625   
answer8     0.999774  0.815172  0.604398  0.378186  0.357774  0.125470   
answer9     0.999602  0.691447  0.201630  0.831034  0.399055  0.074253   
answer10    0.999662  0.822359  0.631726  0.356093  0.400632  0.123542   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                             

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer2     0.970320  0.416934  0.235057  0.581684  0.296060  0.118210   
answer3     0.964914  0.403566  0.165005  0.575299  0.261910  0.213159   
answer4     0.956711  0.572233  0.400077  0.432485  0.341992  0.293737   
answer6     0.946798  0.526863  0.370125  0.414622  0.363524  0.252157   
answer7     0.937989  0.607452  0.403680  0.353418  0.349378  0.189647   
answer8     0.939380  0.495408  0.301801  0.475015  0.276747  0.346247   
answer9     0.948353  0.557861  0.368357  0.376043  0.332483  0.274058   
answer10    0.979444  0.391476  0.262271  0.472143  0.337330  0.195096   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                                             ...                       
answer2   0.233333  0.258499  0.094007  0.733333  ...  0.417978  0.533312   
answer3   0.166667  0.253097

In [24]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2682
Time:                        00:06:28   Log-Likelihood:                -130.95
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 4.936e-13
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.1989      2.677      1.942      0.052      -0.048      10.446
AU01          -9.1031      2.464     -3.694      0.000     -13.933      -4.274
AU02           6.2271

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2732
Time:                        00:06:29   Log-Likelihood:                -130.05
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 2.290e-13
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.6073      2.672      2.099      0.036       0.371      10.843
AU01          -9.2820      2.518     -3.686      0.000     -14.217      -4.347
AU02           5.6205      2.122      2.649      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2711
Time:                        00:06:29   Log-Likelihood:                -126.69
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 1.026e-12
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.5042      2.925      2.907      0.004       2.771      14.238
AU01          -9.8972      2.549     -3.883      0.000     -14.893      -4.901
AU02           6.4444      2.107      3.058      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  271
Model:                          Logit   Df Residuals:                      253
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2975
Time:                        00:06:29   Log-Likelihood:                -127.10
converged:                       True   LL-Null:                       -180.92
Covariance Type:            nonrobust   LLR p-value:                 3.362e-15
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.7226      2.696      2.123      0.034       0.439      11.007
AU01         -10.2116      2.566     -3.979      0.000     -15.241      -5.182
AU02           7.4504      2.140      3.481      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  268
Model:                          Logit   Df Residuals:                      250
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.3426
Time:                        00:06:30   Log-Likelihood:                -114.94
converged:                       True   LL-Null:                       -174.84
Covariance Type:            nonrobust   LLR p-value:                 1.684e-17
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.7246      2.931      1.612      0.107      -1.020      10.469
AU01         -12.4235      2.819     -4.407      0.000     -17.949      -6.898
AU02           5.6287      2.228      2.526      0.0

ファイル名: filtered_ID71.csv
予測率: 70.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 1, 0, 1, 1, 0, 0]

===== テストデータ: ファイル 30/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.3460
Time:                        00:06:30   Log-Likelihood:                -117.68
converged:                       True   LL-Null:                       -179.93
Covariance Type:            nonrobust   LLR p-value:                 2.121e-18
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.2817      2.848      2.206      0.027       0.70

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.44712046832576596, 0.433116686013028, 0.30822007132286056, 0.3183584661895356, 0.39266398316564666, 0.30752788757220983, 0.343943899180851, 0.4775568016243985, 0.46142078873478487, 0.4112957487685299]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31.csv
予測率: 60.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 1, 0, 0, 1, 0, 0, 1]
クラス1の確率: [0.4073361086779696, 0.7922616027936945, 0.3564616519929367, 0.6185379566003171, 0.3051569337614243, 0.352222526916884, 0.5002150148553334, 0.29348295908692645, 0.22050216160186034, 0.53556535597665]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0

最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.31468291467723075, 0.17765882729923002, 0.35902225885672767, 0.34086766750184744, 0.3267094279385038, 0.3479655492961353, 0.2964200611866693, 0.3599448829440796]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.37022914197234597, 0.37128331581110513, 0.367853751293183, 0.36736355541920707, 0.369706113944979, 0.3680657103490941, 0.36780915114858587, 0.36767489752771565]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.4940662796055811, 0.23240766436906846, 0.27303130826129013, 0.28859368724116125, 0.20683570131551474

## statsmodels（先頭10フレームの平均）

In [5]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 平均を計算
    mean_values_per_label = first_10_frames_per_label.groupby("label").mean(numeric_only=True)
    
    # NaNを含む行を削除
    mean_values_per_label = mean_values_per_label.dropna(axis=0)
    
    # Yを追加
    mean_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, mean_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.996606  0.600396  0.295082  0.447557  0.334181  0.174490   0.1   
answer2     0.997146  0.575426  0.358429  0.525853  0.273889  0.171827   0.1   
answer3     0.996037  0.578892  0.316704  0.497932  0.299455  0.160776   0.1   
answer4     0.994471  0.672254  0.385879  0.482804  0.390718  0.150700   0.1   
answer5     0.994645  0.637202  0.329364  0.515847  0.436510  0.156992   0.1   
answer6     0.995012  0.647892  0.478994  0.362251  0.394591  0.158138   0.0   
answer7     0.995345  0.620362  0.375833  0.499724  0.418321  0.203986   0.3   
answer8     0.994680  0.668023  0.363513  0.528893  0.358564  0.158290   0.1   
answer9     0.995906  0.614710  0.362841  0.470365  0.422107  0.167508   0.0   
answer10    0.993830  0.698336  0.482286  0.404548  0.504191  0.169933   0.2   

        

          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.999738  0.739367  0.383070  0.191077  0.388937  0.086635   0.0   
answer2     0.997667  0.645625  0.492112  0.311158  0.419653  0.092321   0.0   
answer3     0.994959  0.689341  0.260078  0.220292  0.411114  0.071542   0.0   
answer4     0.997831  0.528466  0.744082  0.580664  0.303869  0.282081   0.3   
answer5     0.997553  0.497134  0.468842  0.305226  0.374424  0.166276   0.1   
answer6     0.998412  0.451437  0.450278  0.104918  0.480501  0.086521   0.0   
answer7     0.999285  0.631824  0.416685  0.214032  0.392993  0.085851   0.0   
answer8     0.991824  0.626320  0.382052  0.099552  0.490629  0.068328   0.0   
answer9     0.998085  0.563007  0.324008  0.253230  0.518024  0.112277   0.0   
answer10    0.997788  0.829620  0.514755  0.205364  0.341095  0.109058   0.0   

              AU09      AU10  AU11  ...

          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.998954  0.735968  0.515650  0.185040  0.375733  0.081704   0.3   
answer2     0.998966  0.775214  0.585189  0.336720  0.364857  0.166867   0.0   
answer3     0.999337  0.794121  0.638263  0.370939  0.345569  0.130711   0.0   
answer4     0.999362  0.856153  0.664324  0.310743  0.364873  0.144065   0.0   
answer6     0.999051  0.814134  0.716418  0.198047  0.363604  0.147139   0.3   
answer7     0.998667  0.745919  0.528379  0.374851  0.391972  0.100356   0.2   
answer8     0.998713  0.823312  0.691007  0.374731  0.314429  0.082465   0.1   
answer9     0.998714  0.844601  0.756457  0.221839  0.413689  0.179750   0.3   
answer10    0.998711  0.806990  0.743646  0.283312  0.334158  0.223251   0.9   

              AU09      AU10  AU11  ...      AU15      AU17  AU20      AU23  \
label                               ... 

In [26]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2171
Time:                        00:06:37   Log-Likelihood:                -140.10
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 9.866e-10
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0561      2.270      0.025      0.980      -4.394       4.506
AU01          -5.2047      2.080     -2.502      0.012      -9.282      -1.127
AU02           4.3642

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2214
Time:                        00:06:37   Log-Likelihood:                -136.93
converged:                       True   LL-Null:                       -175.86
Covariance Type:            nonrobust   LLR p-value:                 9.196e-10
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.6857      2.348     -0.292      0.770      -5.287       3.916
AU01          -4.8415      2.098     -2.307      0.021      -8.954      -0.729
AU02           4.1575      1.768      2.352      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2252
Time:                        00:06:37   Log-Likelihood:                -138.64
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 3.004e-10
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1186      2.330      0.051      0.959      -4.448       4.685
AU01          -4.4844      2.087     -2.149      0.032      -8.575      -0.394
AU02           3.7872      1.783      2.124      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  271
Model:                          Logit   Df Residuals:                      253
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2729
Time:                        00:06:37   Log-Likelihood:                -129.34
converged:                       True   LL-Null:                       -177.89
Covariance Type:            nonrobust   LLR p-value:                 3.065e-13
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.7608      2.455     -1.125      0.261      -7.573       2.051
AU01          -4.4441      2.147     -2.070      0.038      -8.652      -0.236
AU02           3.3433      1.839      1.818      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  271
Model:                          Logit   Df Residuals:                      253
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2432
Time:                        00:06:38   Log-Likelihood:                -136.91
converged:                       True   LL-Null:                       -180.92
Covariance Type:            nonrobust   LLR p-value:                 1.400e-11
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2619      2.316      0.113      0.910      -4.278       4.802
AU01          -5.8964      2.151     -2.742      0.006     -10.111      -1.681
AU02           5.1259      1.807      2.837      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  268
Model:                          Logit   Df Residuals:                      250
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2264
Time:                        00:06:38   Log-Likelihood:                -135.26
converged:                       True   LL-Null:                       -174.84
Covariance Type:            nonrobust   LLR p-value:                 5.400e-10
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6287      2.331      0.270      0.787      -3.941       5.198
AU01          -4.7486      2.122     -2.238      0.025      -8.908      -0.590
AU02           4.4812      1.800      2.489      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  268
Model:                          Logit   Df Residuals:                      250
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2156
Time:                        00:06:38   Log-Likelihood:                -140.76
converged:                       True   LL-Null:                       -179.44
Covariance Type:            nonrobust   LLR p-value:                 1.123e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0773      2.280     -0.034      0.973      -4.546       4.391
AU01          -4.6851      2.072     -2.262      0.024      -8.745      -0.625
AU02           4.1093      1.760      2.335      0.0

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.43585113743361464, 0.4243130937282209, 0.44742673517371845, 0.36688466582464974, 0.46876633901685355, 0.2778223537400445, 0.4269763459542729, 0.4509384995071862, 0.46585193448967405, 0.35493762204111773]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31.csv
予測率: 70.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 1, 0, 0, 0, 0, 0, 1]
クラス1の確率: [0.39422823646504906, 0.8398972678329334, 0.39894494976063327, 0.541948309277655, 0.3442890758565058, 0.29770932028746494, 0.342622298226252, 0.276176655078814, 0.23619553676931646, 0.5207119304175913]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3699441736952354, 0.3722870643518059, 0.3684408128927288, 0.3702773084037572, 0.37115615079953207, 0.3685798190580754, 0.36824103983601086, 0.36881764909873366]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.48193086391258594, 0.208337969783991, 0.25169485710384903, 0.34178194602538975, 0.23895829428193743, 0.27482306297996695, 0.24984302678946002, 0.2106885387539527, 0.23409653247609125, 0.3241544270790377]

===== テストデータ: ファイル 25/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID63.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.266726071065025, 0.3079009159226063, 0.1163703948024

## statsmodels（先頭30フレームの標準偏差）

In [7]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 標準偏差を計算
    std_values_per_label = first_30_frames_per_label.groupby("label").std(numeric_only=True)

    # NaNを含む行を削除
    std_values_per_label = std_values_per_label.dropna(axis=0)
    
    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.001530  0.066801  0.085971  0.122109  0.076855  0.046814   
answer2     0.000945  0.085073  0.084465  0.104862  0.052543  0.041350   
answer3     0.001572  0.103785  0.125221  0.102060  0.104269  0.032001   
answer4     0.001297  0.072478  0.086068  0.102976  0.114848  0.034969   
answer5     0.001613  0.064449  0.107979  0.117039  0.100585  0.041976   
answer6     0.001492  0.061847  0.100756  0.110784  0.105393  0.027268   
answer7     0.001258  0.062818  0.113542  0.141535  0.163639  0.037189   
answer8     0.001798  0.068074  0.090012  0.107492  0.064596  0.035957   
answer9     0.001176  0.102274  0.099719  0.121681  0.061962  0.044371   
answer10    0.001767  0.060623  0.082212  0.140823  0.071314  0.024458   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
lab

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.000100  0.016419  0.048520  0.144456  0.034798  0.020793   
answer2     0.000063  0.030464  0.062119  0.126147  0.031946  0.017523   
answer3     0.000065  0.013984  0.047985  0.107172  0.025430  0.023022   
answer4     0.000071  0.018927  0.064844  0.113111  0.025002  0.027099   
answer5     0.000286  0.029386  0.076060  0.104377  0.019759  0.013937   
answer6     0.000146  0.027216  0.061307  0.101557  0.022572  0.021898   
answer7     0.000085  0.034467  0.067028  0.122931  0.019091  0.012395   
answer8     0.000199  0.024816  0.029806  0.091416  0.041774  0.005812   
answer9     0.000573  0.017143  0.034355  0.116533  0.022064  0.014453   
answer10    0.001399  0.022334  0.047761  0.100288  0.027376  0.010672   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                             

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer2     0.001650  0.087210  0.129806  0.104157  0.080959  0.065687   
answer3     0.000828  0.108306  0.202201  0.187985  0.056338  0.070089   
answer5     0.000617  0.121396  0.226277  0.217954  0.044153  0.071360   
answer6     0.001093  0.102397  0.153491  0.122961  0.057872  0.079946   
answer7     0.010905  0.086779  0.156016  0.158862  0.101121  0.064739   
answer8     0.000337  0.080729  0.220680  0.216786  0.065356  0.039722   
answer9     0.000426  0.101840  0.114768  0.082961  0.121878  0.070264   
answer10    0.000958  0.113897  0.100768  0.118456  0.090866  0.055858   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                                             ...                       
answer2   0.449776  0.049622  0.121151  0.379049  ...  0.186691  0.055805   
answer3   0.395847  0.106739

In [28]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1763
Time:                        00:06:45   Log-Likelihood:                -147.40
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 3.235e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.8112      0.813     -3.458      0.001      -4.405      -1.218
AU01          -4.7635      5.084     -0.937      0.349     -14.728       5.201
AU02          -4.2491

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1754
Time:                        00:06:45   Log-Likelihood:                -145.01
converged:                       True   LL-Null:                       -175.86
Covariance Type:            nonrobust   LLR p-value:                 5.500e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0153      0.819     -3.680      0.000      -4.621      -1.409
AU01          -5.1153      5.166     -0.990      0.322     -15.240       5.010
AU02          -3.7589      5.606     -0.671      0.5

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1724
Time:                        00:06:45   Log-Likelihood:                -148.09
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 5.518e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.8709      0.815     -3.521      0.000      -4.469      -1.273
AU01          -5.2234      5.166     -1.011      0.312     -15.348       4.902
AU02          -3.8201      5.605     -0.682      0.4

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  271
Model:                          Logit   Df Residuals:                      253
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2360
Time:                        00:06:46   Log-Likelihood:                -135.90
converged:                       True   LL-Null:                       -177.89
Covariance Type:            nonrobust   LLR p-value:                 7.483e-11
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.5072      0.869     -4.037      0.000      -5.210      -1.805
AU01          -5.3379      5.412     -0.986      0.324     -15.946       5.270
AU02          -4.3140      5.899     -0.731      0.4

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  271
Model:                          Logit   Df Residuals:                      253
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2045
Time:                        00:06:46   Log-Likelihood:                -143.92
converged:                       True   LL-Null:                       -180.92
Covariance Type:            nonrobust   LLR p-value:                 4.368e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.5942      0.856     -4.200      0.000      -5.271      -1.917
AU01          -5.5054      5.209     -1.057      0.291     -15.715       4.704
AU02          -5.3490      5.663     -0.944      0.3

ファイル名: filtered_ID63.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 26/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  268
Model:                          Logit   Df Residuals:                      250
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1878
Time:                        00:06:46   Log-Likelihood:                -142.00
converged:                       True   LL-Null:                       -174.84
Covariance Type:            nonrobust   LLR p-value:                 1.180e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.5146      0.854     -4.117      0.000      -5.188      

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1957
Time:                        00:06:46   Log-Likelihood:                -144.71
converged:                       True   LL-Null:                       -179.93
Covariance Type:            nonrobust   LLR p-value:                 1.809e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0040      0.813     -3.695      0.000      -4.598      -1.411
AU01          -8.0366      5.380     -1.494      0.135     -18.582       2.508
AU02           0.8345      5.899      0.141      0.8

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.32963972020608046, 0.33933414205264056, 0.3368380583844367, 0.3006301915756188, 0.3106035594307157, 0.30392856356593045, 0.3491910982001189, 0.3475743545594614, 0.32778487749484636, 0.3218477666068551]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.32350095366525977, 0.4384210701665669, 0.32923736497398437, 0.46094367612369197, 0.35031090994790404, 0.303633298249842, 0.38677326230914233, 0.3258444249447434, 0.2991915689446068, 0.36793971030914663]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 

最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54.csv
予測率: 25.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 1, 0, 0, 1, 0, 0]
クラス1の確率: [0.4044109023719881, 0.3873346297061468, 0.5087091629970432, 0.45900402897612586, 0.39808740059420106, 0.5578029614948109, 0.4437010598833948, 0.39282246304220253]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.41691029630828064, 0.4142531588365251, 0.36447005734014537, 0.41194211355993016, 0.4201663113064257, 0.33515450640391703, 0.3378437255801839, 0.3515679299166148]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3572113018753637, 0.2895577701177337, 0.4170756817302865, 0.3435231959533938, 0.32756688161958913, 0.3

## statsmodels（先頭10フレームの標準偏差）

In [21]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 標準偏差を計算
    std_values_per_label = first_30_frames_per_label.groupby("label").std(numeric_only=True)

    # NaNを含む行を削除
    std_values_per_label = std_values_per_label.dropna(axis=0)
    
    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.001006  0.060736  0.087688  0.092299  0.082711  0.035892   
answer2     0.001069  0.061230  0.085326  0.112583  0.056468  0.046943   
answer3     0.001248  0.108813  0.148984  0.119428  0.065117  0.037821   
answer4     0.001489  0.076634  0.083428  0.071551  0.108240  0.034433   
answer5     0.001968  0.077905  0.102497  0.114414  0.087968  0.041780   
answer6     0.001509  0.054927  0.111568  0.089369  0.089529  0.021638   
answer7     0.001398  0.086977  0.119253  0.127055  0.163408  0.042158   
answer8     0.001651  0.064065  0.075687  0.105997  0.083067  0.034253   
answer9     0.000932  0.113299  0.088482  0.115859  0.074840  0.021622   
answer10    0.002075  0.070181  0.094008  0.116095  0.081840  0.020121   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
lab

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.000148  0.034654  0.062359  0.121993  0.010986  0.026715   
answer2     0.000035  0.075366  0.102713  0.172826  0.044251  0.062337   
answer3     0.000018  0.074797  0.077381  0.114798  0.108030  0.042107   
answer4     0.000143  0.042640  0.118071  0.064415  0.011159  0.037319   
answer5     0.000043  0.076465  0.056014  0.096007  0.075099  0.047829   
answer6     0.000096  0.044624  0.092476  0.070915  0.015076  0.008850   
answer7     0.000033  0.041547  0.058465  0.048190  0.012816  0.056475   
answer8     0.000036  0.034908  0.074444  0.114924  0.018264  0.109952   
answer9     0.000221  0.050425  0.105089  0.057372  0.014511  0.056970   
answer10    0.000060  0.034992  0.087270  0.127980  0.010640  0.029043   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                             

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.000141  0.047400  0.044691  0.046400  0.087316  0.009537   
answer2     0.000307  0.023941  0.036597  0.071554  0.021392  0.010347   
answer3     0.000104  0.029764  0.051967  0.064294  0.044351  0.017602   
answer4     0.000222  0.014843  0.047852  0.089534  0.039542  0.010865   
answer6     0.000171  0.031665  0.026871  0.087880  0.030750  0.023065   
answer7     0.000257  0.141473  0.175538  0.153175  0.032812  0.020782   
answer8     0.000320  0.029032  0.075839  0.121333  0.073414  0.016628   
answer9     0.000177  0.046798  0.076090  0.087985  0.093528  0.039994   
answer10    0.000207  0.029058  0.021708  0.078167  0.030732  0.023247   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                                             ...                       
answer1   0.483046  0.009490  0

In [22]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1556
Time:                        02:43:40   Log-Likelihood:                -151.10
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 5.318e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0438      0.666     -4.572      0.000      -4.349      -1.739
AU01           3.7315      4.411      0.846      0.398      -4.913      12.377
AU02          -6.2374

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1446
Time:                        02:43:41   Log-Likelihood:                -150.43
converged:                       True   LL-Null:                       -175.86
Covariance Type:            nonrobust   LLR p-value:                 3.103e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.2590      0.678     -4.807      0.000      -4.588      -1.930
AU01           2.0336      4.481      0.454      0.650      -6.750      10.817
AU02          -5.7324      4.598     -1.247      0.2

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1547
Time:                        02:43:42   Log-Likelihood:                -151.26
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 6.005e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0133      0.670     -4.497      0.000      -4.327      -1.700
AU01           4.7453      4.536      1.046      0.295      -4.144      13.635
AU02          -8.2657      4.679     -1.767      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1973
Time:                        02:43:43   Log-Likelihood:                -139.52
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 3.788e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.6453      0.703     -5.187      0.000      -5.023      -2.268
AU01           9.7236      4.808      2.022      0.043       0.300      19.147
AU02          -9.0767      4.795     -1.893      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  268
Model:                          Logit   Df Residuals:                      250
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1683
Time:                        02:43:44   Log-Likelihood:                -149.24
converged:                       True   LL-Null:                       -179.44
Covariance Type:            nonrobust   LLR p-value:                 9.049e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.4871      0.679     -5.137      0.000      -4.818      -2.157
AU01           3.2063      4.509      0.711      0.477      -5.631      12.044
AU02          -6.6986      4.654     -1.439      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1438
Time:                        02:43:45   Log-Likelihood:                -150.57
converged:                       True   LL-Null:                       -175.86
Covariance Type:            nonrobust   LLR p-value:                 3.416e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.2306      0.670     -4.821      0.000      -4.544      -1.917
AU01           2.5091      4.478      0.560      0.575      -6.267      11.285
AU02          -5.9641      4.588     -1.300      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  270
Model:                          Logit   Df Residuals:                      252
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1517
Time:                        02:43:46   Log-Likelihood:                -153.05
converged:                       True   LL-Null:                       -180.43
Covariance Type:            nonrobust   LLR p-value:                 7.511e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.1681      0.668     -4.745      0.000      -4.477      -1.859
AU01           3.4849      4.470      0.780      0.436      -5.277      12.246
AU02          -5.9210      4.573     -1.295      0.1

ファイル名: filtered_ID72.csv
予測率: 62.50%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 1, 0, 1, 0, 0, 0]

===== テストデータ: ファイル 31/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  268
Model:                          Logit   Df Residuals:                      250
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1564
Time:                        02:43:47   Log-Likelihood:                -151.38
converged:                       True   LL-Null:                       -179.44
Covariance Type:            nonrobust   LLR p-value:                 4.523e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0893      0.671     -4.601      0.000      -4.405      -1.77

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.28325703438877387, 0.3700334925986962, 0.3895218874017594, 0.3181536181259849, 0.3309411299870573, 0.3226424828828807, 0.38362573910974357, 0.357599964127424, 0.30265678124495665, 0.3028060080949683]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.39099814131506816, 0.3931399357771669, 0.3921778343432035, 0.3908171210229956, 0.39268087649145145, 0.3912438244745365, 0.3919447356656269, 0.391999537751622, 0.3914728433390748, 0.39149650233002714]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0,

最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54.csv
予測率: 62.50%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [1, 0, 1, 1, 1, 1, 0, 0]
クラス1の確率: [0.549122082754874, 0.41641863473390506, 0.5601507465041535, 0.5927791866446043, 0.5125107927660507, 0.62907436695913, 0.4901818077820683, 0.4217158434020799]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56.csv
予測率: 12.50%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [1, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.5142294129325684, 0.40726038399258063, 0.38031494663740123, 0.4833128547879236, 0.39052946680507905, 0.2958483376162832, 0.40086805480173016, 0.34934871010679164]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.35768468707541445, 0.35332341676456736, 0.3564968217580716, 0.3535005372534466, 0.35767980349184464, 0

## statsmodels（先頭30フレームの差分）

In [3]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # ラベルごとにフレーム間の差分を計算
    diff_values_per_label = (
        first_30_frames_per_label.groupby("label")
        .apply(lambda group: group.iloc[:, 1:].diff().dropna())
    )

    # 差分に対応するラベル列をリセット（ラベルを戻す）
    diff_values_per_label = diff_values_per_label.reset_index(level=0).dropna()
    
    # 結果をリストに保存
    result_list.append((file_name, diff_values_per_label))

    # Yを追加
    diff_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
        label  Confidence      AU01      AU02      AU04      AU05      AU06  \
1     answer1    0.002523  0.020600 -0.051877  0.051471 -0.174598 -0.029496   
2     answer1   -0.000777 -0.073554  0.019585 -0.076175  0.157162 -0.012325   
3     answer1   -0.000558  0.192474  0.187477  0.188690 -0.160535 -0.029967   
4     answer1    0.000219 -0.134605  0.036094 -0.201379  0.035716 -0.048250   
5     answer1    0.000912 -0.076099 -0.020339 -0.079744 -0.010129  0.118473   
..        ...         ...       ...       ...       ...       ...       ...   
295  answer10    0.004307 -0.049486 -0.193434  0.265313 -0.097888  0.049583   
296  answer10   -0.000085 -0.013194  0.201371  0.034599 -0.065208 -0.032655   
297  answer10   -0.003561 -0.043562  0.026955 -0.043254  0.008891  0.007323   
298  answer10    0.001567  0.025185 -0.023358 -0.149872 -0.002337  0.024066   
299  answer10    0.001315  0.021714 -0.139229  0.103601 -0.024386 -0.027206   

     AU07      AU09 

        label  Confidence      AU01      AU02      AU04      AU05      AU06  \
1     answer2   -0.001339 -0.217206 -0.340049 -0.292914  0.106526 -0.000227   
2     answer2   -0.003195  0.254189  0.289430  0.152722 -0.033725  0.089664   
3     answer2    0.001316 -0.333534 -0.354853  0.023842 -0.094253 -0.088825   
4     answer2   -0.003646 -0.016174  0.060527  0.086853  0.056385  0.037109   
5     answer2    0.002529  0.240263  0.141640 -0.389780 -0.118589  0.089184   
..        ...         ...       ...       ...       ...       ...       ...   
175  answer10   -0.015456 -0.044894 -0.045234 -0.309510 -0.055658  0.010534   
176  answer10   -0.012070 -0.017432 -0.140615  0.382912 -0.006199  0.081258   
177  answer10    0.039724  0.022565  0.290004 -0.198414  0.127835 -0.018189   
178  answer10   -0.002217 -0.109258 -0.120176  0.079764 -0.101315 -0.074343   
179  answer10   -0.042707 -0.142322 -0.083814 -0.329937  0.012657  0.553408   

     AU07      AU09      AU10  ...      AU15      A

In [4]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 7120
Model:                          Logit   Df Residuals:                     7102
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:               0.0001743
Time:                        14:00:18   Log-Likelihood:                -4762.1
converged:                       True   LL-Null:                       -4762.9
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4455      0.024    -18.334      0.000      -0.493      -0.398
AU01           0.0615      0.266      0.231      0.818      -0.461       0.584
AU02          -0.0140

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 7121
Model:                          Logit   Df Residuals:                     7103
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:               0.0001682
Time:                        14:00:19   Log-Likelihood:                -4762.6
converged:                       True   LL-Null:                       -4763.4
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4457      0.024    -18.345      0.000      -0.493      -0.398
AU01           0.0478      0.263      0.182      0.856      -0.468       0.564
AU02          -0.0116      0.215     -0.054      0.9

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 7149
Model:                          Logit   Df Residuals:                     7131
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:               0.0001773
Time:                        14:00:19   Log-Likelihood:                -4776.4
converged:                       True   LL-Null:                       -4777.2
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4521      0.024    -18.633      0.000      -0.500      -0.405
AU01           0.0412      0.263      0.156      0.876      -0.475       0.557
AU02          -0.0086      0.215     -0.040      0.9

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 7162
Model:                          Logit   Df Residuals:                     7144
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:               0.0001886
Time:                        14:00:19   Log-Likelihood:                -4782.7
converged:                       True   LL-Null:                       -4783.6
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4551      0.024    -18.765      0.000      -0.503      -0.408
AU01           0.0527      0.264      0.200      0.842      -0.464       0.570
AU02          -0.0150      0.216     -0.069      0.9

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 7133
Model:                          Logit   Df Residuals:                     7115
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:               0.0001944
Time:                        14:00:20   Log-Likelihood:                -4621.3
converged:                       True   LL-Null:                       -4622.2
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.6151      0.025    -24.790      0.000      -0.664      -0.567
AU01           0.0627      0.269      0.233      0.815      -0.464       0.589
AU02          -0.0331      0.219     -0.151      0.8

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 7167
Model:                          Logit   Df Residuals:                     7149
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:               0.0001857
Time:                        14:00:20   Log-Likelihood:                -4785.2
converged:                       True   LL-Null:                       -4786.1
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4562      0.024    -18.817      0.000      -0.504      -0.409
AU01           0.0533      0.271      0.197      0.844      -0.478       0.585
AU02          -0.0229      0.219     -0.105      0.9

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 7236
Model:                          Logit   Df Residuals:                     7218
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:               0.0001879
Time:                        14:00:21   Log-Likelihood:                -4818.8
converged:                       True   LL-Null:                       -4819.7
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4718      0.024    -19.520      0.000      -0.519      -0.424
AU01           0.0628      0.267      0.235      0.814      -0.461       0.587
AU02          -0.0225      0.218     -0.103      0.9

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 7183
Model:                          Logit   Df Residuals:                     7165
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:               0.0001769
Time:                        14:00:21   Log-Likelihood:                -4673.4
converged:                       True   LL-Null:                       -4674.3
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5953      0.025    -24.145      0.000      -0.644      -0.547
AU01           0.0525      0.277      0.190      0.849      -0.490       0.595
AU02          -0.0043      0.221     -0.019      0.9

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 7125
Model:                          Logit   Df Residuals:                     7107
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:               0.0001844
Time:                        14:00:22   Log-Likelihood:                -4764.5
converged:                       True   LL-Null:                       -4765.4
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4466      0.024    -18.385      0.000      -0.494      -0.399
AU01           0.0310      0.269      0.115      0.908      -0.496       0.558
AU02          -0.0166      0.217     -0.076      0.9

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID34.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID36.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID38.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

...

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID63.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID69.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID71.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID73.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

## statsmodels（先頭30フレームの差分の標準偏差）

In [9]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # ラベルごとにフレーム間の差分を計算
    diff_values_per_label = (
        first_30_frames_per_label.groupby("label")
        .apply(lambda group: group.iloc[:, 1:].diff().dropna())
    )

    # 差分に対応するラベル列をリセット（ラベルを戻す）
    diff_values_per_label = diff_values_per_label.reset_index(level=0).dropna()
    
    # 差分の標準偏差を計算
    std_values_per_label = (
        diff_values_per_label.groupby("label").std(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.001462  0.094932  0.099262  0.166714  0.088797  0.052320   
answer2     0.001210  0.112730  0.110444  0.150327  0.068664  0.052901   
answer3     0.001548  0.116955  0.136898  0.151067  0.093528  0.041603   
answer4     0.001596  0.097164  0.089056  0.138602  0.157043  0.047489   
answer5     0.002208  0.070803  0.122991  0.145819  0.142746  0.044949   
answer6     0.001575  0.063542  0.092813  0.169444  0.153299  0.035713   
answer7     0.001426  0.089564  0.160565  0.190157  0.215378  0.040423   
answer8     0.001579  0.099095  0.110874  0.151145  0.077755  0.037964   
answer9     0.001348  0.106613  0.140040  0.136975  0.058231  0.043743   
answer10    0.002133  0.072338  0.109438  0.161577  0.077560  0.034574   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
lab

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.000162  0.052917  0.079650  0.135867  0.010279  0.038746   
answer2     0.000050  0.072638  0.099636  0.102774  0.065555  0.053407   
answer3     0.000030  0.074112  0.104587  0.130350  0.078771  0.025848   
answer4     0.000104  0.044179  0.107774  0.087891  0.016033  0.043736   
answer5     0.000047  0.065029  0.080615  0.135514  0.099055  0.048908   
answer6     0.000091  0.060935  0.095431  0.108230  0.041922  0.025289   
answer7     0.000026  0.061097  0.084469  0.054609  0.044559  0.043213   
answer8     0.000039  0.037251  0.099031  0.156913  0.016218  0.132963   
answer9     0.000318  0.065787  0.118832  0.050526  0.027136  0.051560   
answer10    0.000043  0.062765  0.061556  0.147959  0.035372  0.045101   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                             

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1          NaN       NaN       NaN       NaN       NaN       NaN   
answer2     0.014010  0.149169  0.164951  0.260985  0.082113  0.108801   
answer3     0.011212  0.098260  0.109768  0.378475  0.037556  0.083042   
answer4     0.022652  0.189149  0.220116  0.234871  0.113192  0.321114   
answer5          NaN       NaN       NaN       NaN       NaN       NaN   
answer6     0.033940  0.196039  0.312541  0.253986  0.159295  0.352863   
answer7     0.037004  0.105033  0.169319  0.275317  0.117593  0.183930   
answer8     0.030482  0.138490  0.233411  0.410219  0.059153  0.511882   
answer9     0.031634  0.146280  0.152935  0.231732  0.085072  0.158872   
answer10    0.020257  0.111254  0.163059  0.265440  0.083056  0.224924   

              AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                             

In [32]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2167
Time:                        00:07:03   Log-Likelihood:                -140.16
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 1.034e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.9297      0.694     -5.664      0.000      -5.290      -2.570
AU01          -3.2520      4.173     -0.779      0.436     -11.431       4.927
AU02          -5.4079

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2113
Time:                        00:07:04   Log-Likelihood:                -141.13
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 2.283e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.8440      0.695     -5.534      0.000      -5.205      -2.483
AU01          -4.6383      4.195     -1.106      0.269     -12.861       3.584
AU02          -3.7217      4.652     -0.800      0.4

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  268
Model:                          Logit   Df Residuals:                      250
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2137
Time:                        00:07:04   Log-Likelihood:                -141.08
converged:                       True   LL-Null:                       -179.44
Covariance Type:            nonrobust   LLR p-value:                 1.466e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.7836      0.698     -5.422      0.000      -5.151      -2.416
AU01          -4.2318      4.169     -1.015      0.310     -12.403       3.940
AU02          -4.5375      4.633     -0.979      0.3

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  268
Model:                          Logit   Df Residuals:                      250
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2608
Time:                        00:07:04   Log-Likelihood:                -132.65
converged:                       True   LL-Null:                       -179.44
Covariance Type:            nonrobust   LLR p-value:                 1.356e-12
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.9129      0.776     -6.332      0.000      -6.434      -3.392
AU01          -1.4773      4.385     -0.337      0.736     -10.071       7.117
AU02          -5.8223      4.858     -1.199      0.2

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2192
Time:                        00:07:04   Log-Likelihood:                -137.31
converged:                       True   LL-Null:                       -175.86
Covariance Type:            nonrobust   LLR p-value:                 1.250e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.1712      0.706     -5.911      0.000      -5.554      -2.788
AU01          -5.0458      4.274     -1.181      0.238     -13.423       3.332
AU02          -5.0078      4.722     -1.060      0.2

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  270
Model:                          Logit   Df Residuals:                      252
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1980
Time:                        00:07:05   Log-Likelihood:                -141.85
converged:                       True   LL-Null:                       -176.88
Covariance Type:            nonrobust   LLR p-value:                 2.117e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.0778      0.706     -5.776      0.000      -5.461      -2.694
AU01          -3.0474      4.228     -0.721      0.471     -11.334       5.239
AU02          -5.2456      4.679     -1.121      0.2

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.35885804353917605, 0.3654575443390654, 0.3714077627724373, 0.36143705955685773, 0.3598072598040644, 0.35542578783437057, 0.37283980195885486, 0.37226765586842936, 0.3551669715853724, 0.35957159222440127]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3585668349263307, 0.3886856240481452, 0.3687903745404059, 0.38887599259970695, 0.3759727409012516, 0.3584715101044058, 0.3749290364838125, 0.37622736974194887, 0.3621868120200269, 0.37683332369540207]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [

最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54.csv
予測率: 12.50%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 1, 0, 0]
クラス1の確率: [0.408340742278803, 0.37982293523716015, 0.4462568276054225, 0.45435172326332457, 0.40264169934659216, 0.5092716232766109, 0.4231650398582708, 0.40003326250002863]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3750319276132112, 0.39756369982765216, 0.34668938106855096, 0.3692701725253589, 0.36069256400288563, 0.33269952189710117, 0.347657501389732, 0.33803263682908047]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.36865430315271136, 0.33921872334136155, 0.39777020766078136, 0.3619128514638975, 0.4053494993429

## statsmodels（先頭30フレームの最大値）

In [11]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに30フレームの最大値を計算
    max_values_per_label = (
        first_30_frames_per_label.groupby("label").max(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, max_values_per_label))

    # Yを追加
    max_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.998552  0.772197  0.470349  0.833798  0.497938  0.281521   1.0   
answer2     0.998392  0.823720  0.504450  0.698435  0.451498  0.273104   1.0   
answer3     0.998105  0.810446  0.634203  0.735056  0.553234  0.224017   1.0   
answer4     0.996902  0.805873  0.572209  0.587676  0.748750  0.229382   1.0   
answer5     0.997551  0.775502  0.627177  0.667268  0.585380  0.243576   1.0   
answer6     0.997832  0.788442  0.728710  0.603687  0.741321  0.221382   1.0   
answer7     0.997573  0.748430  0.691159  0.728610  0.753132  0.290905   1.0   
answer8     0.997223  0.828781  0.607205  0.764085  0.546507  0.219612   1.0   
answer9     0.998192  0.806971  0.617136  0.706027  0.529929  0.314064   1.0   
answer10    0.996892  0.802406  0.600398  0.747122  0.589219  0.245450   1.0   

        

          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.999707  0.853529  0.671120  0.626889  0.437417  0.217160   1.0   
answer2     0.999853  0.838330  0.657468  0.547944  0.586151  0.306864   1.0   
answer3     0.999880  0.836807  0.668420  0.629941  0.465118  0.257719   1.0   
answer4     0.999697  0.806746  0.697832  0.430664  0.366951  0.312175   1.0   
answer5     0.999862  0.855041  0.634338  0.628105  0.434266  0.482510   1.0   
answer6     0.999857  0.755370  0.707277  0.679619  0.497855  0.181715   1.0   
answer7     0.999890  0.845668  0.740279  0.384596  0.445488  0.419294   1.0   
answer8     0.999883  0.905242  0.749316  0.730291  0.281958  0.715862   1.0   
answer9     0.998598  0.691452  0.603688  0.219252  0.428754  0.384123   1.0   
answer10    0.999787  0.866839  0.584727  0.668449  0.478707  0.313657   1.0   

              AU09      AU10  AU11  ...

          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.999447  0.881979  0.757135  0.349695  0.486276  0.133203   1.0   
answer2     0.999491  0.810152  0.691985  0.488846  0.422756  0.186333   1.0   
answer3     0.999490  0.847472  0.710018  0.646924  0.448757  0.165810   0.0   
answer4     0.999632  0.886289  0.811545  0.454149  0.557214  0.193308   1.0   
answer5          NaN       NaN       NaN       NaN       NaN       NaN   NaN   
answer6     0.999408  0.879227  0.810864  0.407750  0.555311  0.226444   1.0   
answer7     0.999041  0.890419  0.810035  0.585682  0.485060  0.242474   1.0   
answer8     0.999152  0.866776  0.813029  0.649720  0.487708  0.216320   1.0   
answer9     0.999295  0.904163  0.853811  0.389739  0.553984  0.296929   1.0   
answer10    0.998933  0.839453  0.789323  0.389056  0.407714  0.337656   1.0   

              AU09      AU10  AU11  ...

In [34]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1912
Time:                        00:07:12   Log-Likelihood:                -144.72
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 3.994e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.5616      2.585     -1.378      0.168      -8.628       1.504
AU01           1.2489      2.679      0.466      0.641      -4.002       6.500
AU02           2.8369

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1892
Time:                        00:07:12   Log-Likelihood:                -145.09
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 5.363e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0643      2.651     -1.156      0.248      -8.260       2.131
AU01           1.1487      2.704      0.425      0.671      -4.151       6.449
AU02           2.5486      1.570      1.623      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2208
Time:                        00:07:13   Log-Likelihood:                -135.44
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 1.445e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.8957      2.657     -0.713      0.476      -7.104       3.313
AU01           0.9840      2.743      0.359      0.720      -4.393       6.361
AU02           2.4321      1.556      1.563      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1998
Time:                        00:07:13   Log-Likelihood:                -139.08
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 2.680e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.5945      2.713     -0.956      0.339      -7.913       2.724
AU01           1.3981      2.753      0.508      0.612      -3.997       6.794
AU02           3.2215      1.557      2.069      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2328
Time:                        00:07:13   Log-Likelihood:                -133.34
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 2.612e-10
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.6251      2.683     -1.351      0.177      -8.884       1.634
AU01           2.6667      2.807      0.950      0.342      -2.836       8.169
AU02           2.1183      1.588      1.334      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2134
Time:                        00:07:13   Log-Likelihood:                -140.75
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 1.681e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.6256      2.674     -0.982      0.326      -7.867       2.616
AU01           0.3658      2.757      0.133      0.894      -5.038       5.770
AU02           3.5280      1.545      2.283      0.0

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.36143057967772274, 0.3428848964041806, 0.3298526723595998, 0.27264750298904616, 0.3034225387197507, 0.2705734903554805, 0.3350725743433005, 0.3707447541251275, 0.3183266416929922, 0.3167693994274147]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3450226181553408, 0.5599453691203325, 0.3318000012451377, 0.5105656855073822, 0.38131606352364245, 0.32263236995650746, 0.38251036980111974, 0.33377248383959846, 0.3022316928103253, 0.42160123265419164]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 

最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.4117214987844628, 0.30990884811108355, 0.4818776913532641, 0.43747747403855775, 0.4195047208878784, 0.4357227952060878, 0.4369712019584013, 0.3656901378435723]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.4854504312062347, 0.41986529339241424, 0.40413929990186814, 0.4323769037371329, 0.4300726946197153, 0.4395762491958736, 0.4112819644992032, 0.4017946825544024]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3634892874348063, 0.2736319757974248, 0.4684665311959354, 0.33971098003731703, 0.27989626686064806, 

## statsmodels（先頭10フレームの最大値）

In [13]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに10フレームの最大値を計算
    max_values_per_label = (
        first_10_frames_per_label.groupby("label").max(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, max_values_per_label))

    # Yを追加
    max_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.998552  0.740146  0.404279  0.597930  0.460274  0.214406   1.0   
answer2     0.998392  0.685272  0.488080  0.692978  0.420650  0.256867   1.0   
answer3     0.998105  0.737937  0.634203  0.735056  0.459632  0.214803   1.0   
answer4     0.996902  0.772896  0.464763  0.581556  0.555469  0.215759   1.0   
answer5     0.996959  0.757068  0.453117  0.667268  0.574917  0.243576   1.0   
answer6     0.997096  0.743668  0.728710  0.480454  0.553200  0.191910   0.0   
answer7     0.997573  0.748430  0.528374  0.656478  0.752446  0.290905   1.0   
answer8     0.997107  0.749335  0.464386  0.738110  0.546507  0.218950   1.0   
answer9     0.996756  0.806971  0.539072  0.625298  0.522182  0.207981   0.0   
answer10    0.996236  0.802406  0.600398  0.590113  0.551615  0.214269   1.0   

        

          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.999587  0.881700  0.832166  0.771582  0.392838  0.133460   0.0   
answer2     0.999532  0.844189  0.807780  0.834830  0.349488  0.130433   0.0   
answer3     0.999523  0.845063  0.767871  0.777881  0.318442  0.153159   0.0   
answer4     0.999571  0.845495  0.744589  0.846508  0.281953  0.164390   0.0   
answer5     0.998475  0.816425  0.672744  0.819058  0.337624  0.111901   0.0   
answer6     0.999320  0.825366  0.641001  0.768939  0.325131  0.172718   0.0   
answer7     0.999403  0.798194  0.628615  0.769943  0.323167  0.083616   0.0   
answer8     0.999359  0.930646  0.865970  0.551917  0.381930  0.074709   0.0   
answer9     0.997164  0.872280  0.797989  0.834111  0.327198  0.121272   0.0   
answer10    0.995462  0.888607  0.819811  0.607229  0.396025  0.079421   0.0   

              AU09      AU10  AU11  ...

In [36]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2092
Time:                        00:07:20   Log-Likelihood:                -141.50
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 3.066e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -7.1046      2.200     -3.229      0.001     -11.417      -2.792
AU01           2.6217      2.283      1.148      0.251      -1.853       7.096
AU02           1.5030

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2057
Time:                        00:07:21   Log-Likelihood:                -142.12
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 5.071e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.5416      2.221     -2.946      0.003     -10.894      -2.189
AU01           2.5494      2.330      1.094      0.274      -2.016       7.115
AU02           1.1367      1.370      0.830      0.4

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2167
Time:                        00:07:21   Log-Likelihood:                -136.15
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 2.576e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.8824      2.231     -2.188      0.029      -9.256      -0.509
AU01           1.8038      2.272      0.794      0.427      -2.650       6.258
AU02           1.3055      1.314      0.993      0.3

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2095
Time:                        00:07:21   Log-Likelihood:                -137.40
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 7.033e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.4141      2.295     -2.794      0.005     -10.913      -1.915
AU01           3.6506      2.397      1.523      0.128      -1.047       8.349
AU02           1.7511      1.335      1.312      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2417
Time:                        00:07:22   Log-Likelihood:                -131.79
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 7.304e-11
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -7.8169      2.390     -3.270      0.001     -12.502      -3.132
AU01           3.8039      2.503      1.519      0.129      -1.103       8.710
AU02           1.4211      1.379      1.030      0.3

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  270
Model:                          Logit   Df Residuals:                      252
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1953
Time:                        00:07:22   Log-Likelihood:                -142.33
converged:                       True   LL-Null:                       -176.88
Covariance Type:            nonrobust   LLR p-value:                 3.080e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.8021      2.217     -3.068      0.002     -11.147      -2.457
AU01           2.6045      2.294      1.135      0.256      -1.892       7.101
AU02           1.6557      1.304      1.270      0.2

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.32910528506504444, 0.3767388144354551, 0.37947584489288666, 0.2944657161349817, 0.34519138402057536, 0.2935331910386342, 0.365525903202815, 0.37051211487349733, 0.31416236794090485, 0.29248264766827003]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3798428374874245, 0.6219178301509891, 0.33992480831722066, 0.3888689856766046, 0.3869873216390595, 0.3394985426527165, 0.436216402547618, 0.35478600214426487, 0.30474731129413446, 0.41302680192666685]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0,

最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54.csv
予測率: 12.50%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 1, 0, 0]
クラス1の確率: [0.460021256793361, 0.3570164100123826, 0.47950805321236173, 0.49479414142356953, 0.46632905031235095, 0.5018684698417905, 0.43109465784644874, 0.3794022080334705]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56.csv
予測率: 12.50%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [1, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.5219732453881711, 0.44056220148839165, 0.4249635945799772, 0.4799515172163156, 0.4301779527362562, 0.3926443778076549, 0.42005293612115396, 0.4357796890814182]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.4092457129823009, 0.2603121530199509, 0.3974398202009484, 0.26842879349023563, 0.3252397133250323

## statsmodels（先頭30フレームの最大値と最小値の差分）

In [15]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに最大値を計算
    max_values_per_label = (
        first_30_frames_per_label.groupby("label").max(numeric_only=True)
    )

    # ラベルごとに最小値を計算
    min_values_per_label = (
        first_30_frames_per_label.groupby("label").min(numeric_only=True)
    )

    # 最大値と最小値の差分を計算
    range_values_per_label = max_values_per_label - min_values_per_label

    
    # 結果をリストに保存
    result_list.append((file_name, range_values_per_label))

    # Yを追加
    range_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.006316  0.254691  0.337912  0.633875  0.265135  0.187152   1.0   
answer2     0.003334  0.360066  0.284054  0.404317  0.227819  0.160198   1.0   
answer3     0.005948  0.412699  0.485027  0.423869  0.311095  0.129310   1.0   
answer4     0.004306  0.272166  0.359251  0.414689  0.485407  0.142035   1.0   
answer5     0.006226  0.248887  0.445189  0.463946  0.334733  0.152792   1.0   
answer6     0.005584  0.225375  0.392590  0.401407  0.457584  0.112775   1.0   
answer7     0.005535  0.268238  0.529722  0.553741  0.478301  0.185603   1.0   
answer8     0.006327  0.262135  0.358655  0.426853  0.292341  0.105205   1.0   
answer9     0.004578  0.412329  0.461215  0.456385  0.231980  0.178472   1.0   
answer10    0.006379  0.234590  0.290516  0.518986  0.305036  0.095292   1.0   

        

          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.008459  0.172634  0.502612  0.612089  0.311990  0.368043   1.0   
answer2     0.008997  0.160124  0.274518  0.558122  0.070187  0.177843   1.0   
answer3     0.000651  0.084976  0.191912  0.511979  0.086295  0.253910   1.0   
answer4     0.001298  0.093185  0.184569  0.307482  0.073581  0.101347   0.0   
answer5     0.000714  0.126531  0.158280  0.295179  0.106364  0.262682   1.0   
answer6          NaN       NaN       NaN       NaN       NaN       NaN   NaN   
answer7     0.001553  0.144362  0.269699  0.375377  0.139169  0.357363   1.0   
answer8     0.001417  0.115586  0.270919  0.355899  0.117153  0.370838   1.0   
answer9     0.000275  0.126838  0.195530  0.353804  0.026997  0.192926   1.0   
answer10    0.000309  0.209032  0.206461  0.332169  0.122579  0.519582   1.0   

              AU09      AU10  AU11  ...

          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.001137  0.211262  0.296994  0.239070  0.252264  0.083415   1.0   
answer2     0.001163  0.162327  0.172458  0.308833  0.100709  0.046376   1.0   
answer3     0.000434  0.107012  0.155187  0.451083  0.155635  0.057604   0.0   
answer4     0.000894  0.135427  0.216919  0.360255  0.251771  0.095864   1.0   
answer5          NaN       NaN       NaN       NaN       NaN       NaN   NaN   
answer6     0.000933  0.134543  0.224724  0.348959  0.250457  0.132457   1.0   
answer7     0.001446  0.530376  0.544167  0.501879  0.175295  0.171085   1.0   
answer8     0.001105  0.109644  0.255243  0.565022  0.253536  0.152690   1.0   
answer9     0.000944  0.192315  0.312863  0.311651  0.254819  0.181243   1.0   
answer10    0.001157  0.225127  0.217324  0.279061  0.141464  0.149856   1.0   

              AU09      AU10  AU11  ...

In [38]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1699
Time:                        00:07:29   Log-Likelihood:                -148.53
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 7.714e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.6380      0.909     -2.904      0.004      -4.419      -0.857
AU01           0.0396      1.547      0.026      0.980      -2.993       3.072
AU02          -1.9564

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1808
Time:                        00:07:30   Log-Likelihood:                -146.59
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 1.725e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.6192      0.924     -2.834      0.005      -4.431      -0.808
AU01           0.7273      1.589      0.458      0.647      -2.387       3.842
AU02          -2.8000      1.680     -1.667      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2038
Time:                        00:07:30   Log-Likelihood:                -138.38
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 1.540e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.6581      0.976     -3.746      0.000      -5.572      -1.744
AU01           1.5426      1.630      0.946      0.344      -1.652       4.738
AU02          -2.1159      1.682     -1.258      0.2

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1690
Time:                        00:07:30   Log-Likelihood:                -144.44
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 1.693e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.6872      0.923     -2.911      0.004      -4.497      -0.878
AU01          -0.1154      1.577     -0.073      0.942      -3.206       2.975
AU02          -1.4828      1.653     -0.897      0.3

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2201
Time:                        00:07:30   Log-Likelihood:                -135.56
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 1.591e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.0857      0.976     -2.138      0.033      -3.998      -0.174
AU01          -2.1457      1.754     -1.223      0.221      -5.584       1.293
AU02          -3.2219      1.779     -1.811      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2048
Time:                        00:07:31   Log-Likelihood:                -142.28
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 5.766e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.4450      0.950     -2.572      0.010      -4.308      -0.582
AU01           0.7148      1.599      0.447      0.655      -2.419       3.848
AU02          -1.9020      1.672     -1.137      0.2

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3518103115752214, 0.3295992377544854, 0.3392334344288861, 0.2948361278930978, 0.3048216717508903, 0.28995480800767826, 0.36094310814836633, 0.3517599320272929, 0.33043009796146033, 0.3127807229276991]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.294939844604015, 0.4439965349661109, 0.32019336092557293, 0.47422359059996777, 0.37188235682420956, 0.3022114513577646, 0.3694903073809342, 0.31896907565047977, 0.29810628805801903, 0.39184653602669206]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0

最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54.csv
予測率: 12.50%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 1, 0, 0, 0, 0, 0]
クラス1の確率: [0.4592459274143645, 0.3380024077584963, 0.5214428179079577, 0.4917202964559364, 0.45867620386212016, 0.4865951303622235, 0.4867378115495309, 0.40335198722792004]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.46772228679370503, 0.4011876324594556, 0.36122421147486494, 0.41280841527920203, 0.4312703148515585, 0.3861667383345262, 0.3911115503355122, 0.35429074619024775]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.33383401159577, 0.27476174618694793, 0.4830775836432439, 0.3509531914804018, 0.2918100601237461, 

## xgboost（先頭10フレームの最大値と最小値の差分）

In [17]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに最大値を計算
    max_values_per_label = (
        first_10_frames_per_label.groupby("label").max(numeric_only=True)
    )

    # ラベルごとに最小値を計算
    min_values_per_label = (
        first_10_frames_per_label.groupby("label").min(numeric_only=True)
    )

    # 最大値と最小値の差分を計算
    range_values_per_label = max_values_per_label - min_values_per_label

    
    # 結果をリストに保存
    result_list.append((file_name, range_values_per_label))

    # Yを追加
    range_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1     0.003534  0.210704  0.243156  0.281124  0.197569  0.120037   1.0   
answer2     0.003334  0.221618  0.229155  0.302576  0.196970  0.143961   1.0   
answer3     0.004746  0.340190  0.485027  0.354739  0.217492  0.120096   1.0   
answer4     0.004306  0.226259  0.251805  0.227426  0.292127  0.123546   1.0   
answer5     0.005634  0.230452  0.271130  0.284695  0.303500  0.133120   1.0   
answer6     0.004848  0.180600  0.392590  0.277911  0.269463  0.070650   0.0   
answer7     0.004615  0.268238  0.366938  0.329792  0.477615  0.133785   1.0   
answer8     0.004237  0.182690  0.215836  0.400878  0.274844  0.101990   1.0   
answer9     0.003143  0.343358  0.277812  0.288465  0.212358  0.072390   0.0   
answer10    0.005723  0.234590  0.243214  0.361977  0.267432  0.064111   1.0   

        

          Confidence      AU01      AU02      AU04      AU05      AU06  AU07  \
label                                                                          
answer1          NaN       NaN       NaN       NaN       NaN       NaN   NaN   
answer2     0.001216  0.108677  0.248423  0.227798  0.178688  0.090982   1.0   
answer3     0.000764  0.113859  0.279786  0.224266  0.203595  0.041078   0.0   
answer4     0.001580  0.118829  0.167165  0.411894  0.252672  0.114880   0.0   
answer5     0.002852  0.091222  0.217279  0.260366  0.205221  0.091130   1.0   
answer6          NaN       NaN       NaN       NaN       NaN       NaN   NaN   
answer7     0.005499  0.291400  0.251928  0.323204  0.206132  0.038589   1.0   
answer8          NaN       NaN       NaN       NaN       NaN       NaN   NaN   
answer9          NaN       NaN       NaN       NaN       NaN       NaN   NaN   
answer10    0.000878  0.164463  0.439344  0.293028  0.168448  0.132564   1.0   

              AU09      AU10  AU11  ...

In [40]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1498
Time:                        00:07:38   Log-Likelihood:                -152.13
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 1.143e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.1482      0.684     -4.602      0.000      -4.489      -1.807
AU01           1.8006      1.517      1.187      0.235      -1.172       4.773
AU02          -2.5586

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1416
Time:                        00:07:38   Log-Likelihood:                -150.96
converged:                       True   LL-Null:                       -175.86
Covariance Type:            nonrobust   LLR p-value:                 4.548e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.4274      0.698     -4.908      0.000      -4.796      -2.059
AU01           1.0970      1.544      0.710      0.477      -1.929       4.123
AU02          -2.3155      1.450     -1.597      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1508
Time:                        00:07:38   Log-Likelihood:                -151.96
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 1.007e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0907      0.691     -4.470      0.000      -4.446      -1.736
AU01           1.8866      1.543      1.223      0.221      -1.137       4.910
AU02          -3.1155      1.475     -2.113      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1964
Time:                        00:07:39   Log-Likelihood:                -139.68
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 4.284e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.8703      0.726     -5.331      0.000      -5.293      -2.447
AU01           3.8309      1.633      2.345      0.019       0.630       7.032
AU02          -3.3078      1.510     -2.190      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1299
Time:                        00:07:39   Log-Likelihood:                -153.02
converged:                       True   LL-Null:                       -175.86
Covariance Type:            nonrobust   LLR p-value:                 0.0001927
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.1373      0.690     -4.549      0.000      -4.489      -1.786
AU01           1.7471      1.528      1.144      0.253      -1.247       4.741
AU02          -2.1726      1.443     -1.505      0.1

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  268
Model:                          Logit   Df Residuals:                      250
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1677
Time:                        00:07:39   Log-Likelihood:                -145.52
converged:                       True   LL-Null:                       -174.84
Covariance Type:            nonrobust   LLR p-value:                 1.763e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.9411      0.740     -5.327      0.000      -5.391      -2.491
AU01           1.1793      1.590      0.742      0.458      -1.936       4.295
AU02          -2.9383      1.516     -1.939      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1575
Time:                        00:07:40   Log-Likelihood:                -151.59
converged:                       True   LL-Null:                       -179.93
Covariance Type:            nonrobust   LLR p-value:                 3.653e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.3958      0.687     -4.946      0.000      -4.741      -2.050
AU01           1.0597      1.550      0.684      0.494      -1.978       4.097
AU02          -1.1579      1.555     -0.745      0.4

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.36776671576320885, 0.3897777991223676, 0.3939648234057491, 0.37965204679126013, 0.37643869550605097, 0.3777026786399951, 0.3872938905788424, 0.38311654950360263, 0.3706813196936689, 0.3747256243831296]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.37712187812692094, 0.39398408971408283, 0.38436241977141483, 0.3759143431798007, 0.39122851954443827, 0.3827183765017249, 0.38557677628629833, 0.3829380967413509, 0.3776230873429161, 0.3792059671506009]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル

最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56.csv
予測率: 12.50%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [1, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.5280031116509714, 0.4142510902707096, 0.3693906554321012, 0.4908213700063433, 0.3826536152644477, 0.30226339916463996, 0.41524491741633673, 0.35379401649549275]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.36763469476316624, 0.3364255038796451, 0.3616673835939274, 0.33752598565390296, 0.366362087317672, 0.36336317698037346, 0.3557295008120439, 0.3662627174481171, 0.3673972174895607, 0.3887803218424732]

===== テストデータ: ファイル 25/31 =====
最適パラメータ: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID63.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.2584521446298882, 0.23836682829258743, 0.2142338614